In [3]:
import pandas as pd
import numpy as np

In [4]:
# 1. Cargar los archivos

df_autogluon = pd.read_csv("predicciones_febrero2020_01-07-25_2.csv")
df_promedios = pd.read_csv("promedio_12m_febrero2020_2.csv")


product_preciso = pd.read_csv("product_id_promedio_preciso.csv")

In [5]:
# Reemplazar las 'tn' en df_autogluon por las de df_promedios para los productos en product_preciso

# Obtener los product_id precisos
ids_precisos = product_preciso["product_id"].tolist()

# Crear un mapeo de tn desde df_promedios para los ids precisos
tn_promedios_precisos = df_promedios.set_index("product_id").loc[ids_precisos, "tn"]

# Reemplazar en df_autogluon
df_autogluon.loc[df_autogluon["product_id"].isin(ids_precisos), "tn"] = \
    df_autogluon.loc[df_autogluon["product_id"].isin(ids_precisos), "product_id"].map(tn_promedios_precisos)

# Verificar reemplazo
print(df_autogluon[df_autogluon["product_id"].isin(ids_precisos)].head())

    product_id          tn
8        20009  541.322587
10       20011  377.635440
14       20015  369.023652
17       20018  275.445111
26       20027  230.014917


In [6]:
df_autogluon[["product_id", "tn"]].to_csv("df_autogluon_salida_190725_1.csv", index=False)

In [ ]:



# 2. Calcular errores absolutos para comparar métodos
comparacion_febrero["error_autogluon"] = abs(comparacion_febrero["tn"] - comparacion_febrero["tn_feb_2019"])
comparacion_febrero["error_promedio"] = abs(comparacion_febrero["tn"] - comparacion_febrero["promedio_ultimos_12m"])

# 3. Separar productos según qué método fue mejor
mejor_con_promedio = comparacion_febrero[comparacion_febrero["error_promedio"] < comparacion_febrero["error_autogluon"]]["product_id"].tolist()
mejor_con_autogluon = comparacion_febrero[comparacion_febrero["error_autogluon"] <= comparacion_febrero["error_promedio"]]["product_id"].tolist()

# 4. Tomar predicciones de febrero 2020 según el mejor método para cada grupo
preds_autogluon = df_autogluon[df_autogluon["product_id"].isin(mejor_con_autogluon)].copy()
preds_promedio = df_promedios[df_promedios["product_id"].isin(mejor_con_promedio)].copy()

# 5. Renombrar columna a 'tn_pred_ensemble' en ambos casos
preds_autogluon = preds_autogluon.rename(columns={"tn": "tn_pred_ensemble"})
preds_promedio = preds_promedio.rename(columns={"tn": "tn_pred_ensemble"})

# 6. Unir ambos grupos
ensemble_df = pd.concat([preds_autogluon[["product_id", "tn_pred_ensemble"]],
                         preds_promedio[["product_id", "tn_pred_ensemble"]]],
                        axis=0).sort_values("product_id").reset_index(drop=True)

# 7. Verificación final
print(f"Número total de product_id en ensemble: {ensemble_df['product_id'].nunique()}")
print(ensemble_df.head())

# 8. Guardar a CSV si querés
ensemble_df.to_csv("ensemble_febrero2020_190725_1.csv", index=False)


Número total de product_id en ensemble: 655
   product_id  tn_pred_ensemble
0       20001       1289.854612
1       20002       1043.714777
2       20003        730.515933
3       20004        542.798590
4       20005        534.809802
